In [32]:
import contractions
import nltk
import pandas as pd
import re
import string

from collections import Counter
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer

In [33]:
df = pd.read_csv('Downloads/Datasets/candidates_tweets.csv') #raw tweet data

df.rename(columns={'text': 'original_tweets'}, inplace=True) #retain originals

In [34]:


def process_tweet_text(text):
    """cleans data for clarity and uniformity"""
    text = re.sub(r"http\S+", "", text)
    text = contractions.fix(text)
    text = text.replace('\n', ' ')
    text = text.replace('—', ' ') # two types of hyphens
    text = text.replace('-', ' ')
    text = re.sub('@[A-Za-z0-9_]\S+', '', text)
    text = re.sub('#[A-Za-z0-9_]\S+', '', text)
    text = ''.join([c for c in text if c not in string.punctuation])
    text = re.sub('[^a-zA-Z0-9 -]', '', text)
    return text


df['processed_tweets'] = df['original_tweets'] # copy tweet text over to a new column and then clean the text of the entire column
df['processed_tweets'] = df['processed_tweets'].apply(lambda x: process_tweet_text(x))

In [36]:
#df.to_csv('processed_canditate_tweets.csv') #save results for the record before I mess with the text contents

In [37]:
# break up primary dataframe into individual datframes
JoeBiden = df.loc[df['name'] == 'JoeBiden']
ElizabethWarren = df.loc[df['name'] == 'ewarren']
BernieSanders = df.loc[df['name'] == 'BernieSanders']
MikeBloomberg = df.loc[df['name'] == 'MikeBloomberg']
PeteButtigieg = df.loc[df['name'] == 'PeteButtigieg']
AmyKlobuchar = df.loc[df['name'] == 'amyklobuchar']
DonaldTrump = df.loc[df['name'] == 'realDonaldTrump']
Democrats = df.loc[df['name'] != 'realDonaldTrump']

In [38]:


def tokenize_ngrams(candidate):
    
    """This tokenizes the text as well as calculates ngrams to find common phrases"""
    
    # put all of the tweets together in an object called 'text_all'
    text_all = candidate['processed_tweets'].str.cat(sep=' ')

    # intialize tokenizer and tokenize all of the text
    tokenizer = RegexpTokenizer('\s+', gaps=True)
    tokens = tokenizer.tokenize(text_all.lower())

    # eliminate stop words from text
    cached_stop_words = stopwords.words("english")
    filtered_tokens = [w for w in tokens if not w in cached_stop_words] 
    
    # find most common phrases of 2 & 3 words and store them together in ngrams
    bi_gram = Counter(list(ngrams(filtered_tokens, 2)))
    tri_gram = Counter(list(ngrams(filtered_tokens, 3)))
    ngram = bi_gram.most_common(25) + tri_gram.most_common(25)

    # reate dataframe to store data
    results = pd.DataFrame(ngram, columns=['phrase', 'count'])
    
    return results



In [39]:
# loop through the tweets of each candidate and create a 
# combined dateframe representing the most common phrases
# for each political candidate.

handles = [JoeBiden, ElizabethWarren, BernieSanders, 
           MikeBloomberg, PeteButtigieg, AmyKlobuchar,
           Democrats, DonaldTrump]

results_df = pd.DataFrame()

for handle in handles:
    df_new = tokenize_ngrams(handle)
    results_df = pd.concat((results_df, df_new))
    
    
# reformat tuple into a clean string for formatting purposes
def convert_tuple(tup): 
    str =  ' '.join(tup) 
    str = '"' + str + '"'
    return str


results_df['phrase'] = results_df['phrase'].apply(lambda x: convert_tuple(x)) #apply to each row    

In [ ]:
# results_df.to_csv('most_common_phrases.csv')